Def fonction : à l’intérieur on dit le type des argumetns ; et dire l format de sortie (df, liste, …) ; décrire les arguments de la fonction

In [ ]:
### Define a function to prepare input data for a given start date 
def prepare_input_data(start_date, batch=True, batching_time=8) : 
    """
            Arguments
            ---------
                start_date: datetime
                    a speficic date to get orders
                batch : bool
                    specify if we are batching or not. In case of mono dispatch, the most recent order will be consider.
                batching_time : int
                    batching time used to collect all orders and drivers from start_date to start_date +  batching_time
            
            Return
            ------- 
                list of nearby orders and drivers dataframe to create json inputs file (for Nextmv and LogisticsOS)
    """
    ## define batching end date. Re-format datetime format
    start_date_str = pd.to_datetime(start_date).strftime("%Y-%m-%d %H:%M:%S")

    ## In case of batching 
    if batch==True:
        batching_time_end =  start_date + datetime.timedelta(minutes=batching_time)
        batching_time_end_str = pd.to_datetime(batching_time_end).strftime("%Y-%m-%d %H:%M:%S")
        data = orders.loc[(orders['order_time_cvrt']>= start_date_str) & (orders['order_time_cvrt'] <=batching_time_end_str)]

    ## In case of mono dispatch
    else:
        last_order = orders.loc[orders['order_time_cvrt']<= start_date_str]['order_id'].iloc[-1]
        data = orders.loc[orders['order_id']==last_order]

    ## Get all orders
    orders_list =  data.order_id.unique().tolist()

    ## list which will contain driver locations and ongoing orders
    drivers_input = []
    ## For each order, get the list of nearby drivers. For each nearby driver, retrieve ongoing order
    ## Ongoing order : order previously assigned to the driver (before the run) and which is not yet delivered at the run time.
    ## For orders input : return order id and time, delivery address (lat, lng), restaurant adress (lat, lng)
    ## For drivers input : return driver id, location and timestamp, ongoing orders (order id and time, assigned at, delivered at)
    for order in orders_list :
        drivers, nearby_drivers_table = nearby_drivers(order, start_date, batch, batching_time)
        nearby_drivers_orders = orders.loc[orders['driverid'].isin(drivers)].drop_duplicates('order_id')
        ongoing_orders = nearby_drivers_orders.loc[(nearby_drivers_orders['final_assigned_at']<= start_date_str) & (nearby_drivers_orders['food_delivered']>= start_date_str)].drop_duplicates('order_id')[['driverid', 'order_id', 'order_time', 'final_assigned_at', 'food_delivered']].sort_values('driverid')
        ongoing_orders.columns = ['driverid','ongoing_order_id', 'ongoing_order_time',	'ongoing_order_final_assigned_at',	'ongoing_order_delivered_at']
        ongoing_orders.ongoing_order_id = ongoing_orders.ongoing_order_id.fillna('')
        drivers_table = nearby_drivers_table.merge(ongoing_orders, on ='driverid', how='left')
        drivers_input.append(drivers_table)

    orders_table = data[['order_id','order_time', 'delivery_address_lat', 'delivery_address_lng','shopid','restaurant_lat', 'restaurant_lng']].drop_duplicates('order_id').sort_values('order_time')
    drivers_input = pd.concat(drivers_input).sort_values(['received_timestamp'], ascending=False).drop_duplicates('driverid')
    backlog_input =  orders.loc[orders['order_id'].isin(drivers_input.ongoing_order_id.unique())][['order_id','order_time', 'delivery_address_lat', 'delivery_address_lng','shopid','restaurant_lat', 'restaurant_lng']].drop_duplicates('order_id').sort_values('order_time')
    return (drivers_input, orders_table, backlog_input)